# Read TileWall data and convert it to multple .json files


*I took the data from the google doc on the SVL drive, and converted it first to csv files.  Here I'm reading this in to create individual json files for each line.  I will write another code that will compile each individual json file into one json file to be read into the app.  I think this may be the easiest method for others to add additional objects.*

In [1]:
import pandas as pd
import numpy as np
import os
import wikipedia
import wptools #https://github.com/siznax/wptools

###  A function to grab an image from Wikipedia

*https://wikipedia.readthedocs.io/en/latest/quickstart.html*

*Updated below to match the ../addObject.ipynb code*

In [2]:
def getCaption(parsetree,img, maxLen = 1000, minLen = 10):
    p1 = 0
    if ('/' in img):
        p1 = img.rfind('/')+1
    imgSearch = img[p1:].replace('_',' ')
    print("\nSEARCHING FOR : ", img, imgSearch)

    #print("PARSETREE :",parsetree)
    caption = ''
    p1 = parsetree.lower().find(imgSearch.lower())
    if (p1 >= 0):
        caption = parsetree[p1+len(imgSearch):]
        #get the full caption, with all the markup
        left = 2
        right = 0
        for i,c in enumerate(caption):
            if (c == '['):
                left += 1
            if (c == ']'):
                right += 1
            if (left == right):
                break
        caption = caption[0:i-1]
        #print("first pass : ",caption)
        #remove the markup
        #for citations
        Ntrial = 0
        while ('{{' in caption and Ntrial < 100): 
            remove = []
            p1 = caption.find('{{')
            p2 = caption.find('}}')
            remove.extend([i+p1+1 for i in range(p2-p1)])
            cap = ''
            for i, c in enumerate(caption):
                if (i not in remove):
                    cap += c
            #print('cap : ', cap, remove)
            caption = cap
            Ntrial += 1
        
        #links to other wikipedia pages
        Ntrial = 0
        while ('[[' in caption and Ntrial < 100): 
            remove = []
            p1 = caption.find('[[')
            p2 = caption.find(']]')
            remove.extend([p1,p1+1, p2,p2+1])
            check = caption[p1:p2]
            #print("checking", check)
            p3 = check.find('|')
            if (p3 > 0):
                remove.extend([i+p1+1 for i in range(p3)])
            cap = ''
            for i, c in enumerate(caption):
                if (i not in remove):
                    cap += c
            #print('cap : ', cap, remove)
            caption = cap
            Ntrial += 1
            
        #other html markup
        Ntrial = 0
        done = False
        #print('caption to here : ', caption)
        while (not done and Ntrial < 100):
            remove = []
            p1 = caption.find('<ext>')
            p2 = caption.find('</ext>')
            if (p1 >=0 and p2 >=0):
                remove.extend([i+p1 for i in range(p2-p1+6)])
            else:
                done = True
                break
            cap = ''
            for i, c in enumerate(caption):
                if (i not in remove):
                    cap += c
            #print('cap : ', cap, remove)
            caption = cap
            Ntrial += 1
            
        #remove any bits at the beginning
        p1 = 0
        p = caption.rfind('|')
        if (p > 0):
            p1 = p
        caption = caption[p1+1:]
        
        
        if (len(caption) > maxLen or len(caption) < minLen):
            print("bad caption : ", len(caption))
            caption = None
            
    print("CAPTION : ", caption)
        
        
    return caption


def getWikiImages(nme, category):
    #https://wikipedia.readthedocs.io/en/latest/quickstart.html
    nme = nme.replace('(',' ')
    nme = nme.replace(')',' ')
    if (nme[0] == "M" and nme[1].isdigit()):
        nme_split = nme[1:].split(' ')
        nme = '"Messier '+nme_split[0]+'"'
        for i, x in enumerate(nme_split):
            if (i>0):
                nme+=' '+x
    if (category == "Galaxies"):
        #category = "Galaxy"
        category = ""
    if (category == "Galaxy Clusters"):
       #category = "Galaxy Cluster"
        category = ""
    if (category == "Nebulae"):
        #category = "Nebula"
        category = ""
    if (category == "Star Clusters"):
        #category = "Star Cluster"
        category = ""
    if (category == "Stars"):
        category = "Star"
    if (category == "Constellations"):
        category = "Constellation"
    
    #maybe this will work more consistently?
    if (nme[0:7] == "Messier"):
        catetory = ""

    #search = nme + " " + '"'+category+'"' + " astronomy"
    search = nme + " " + category# + " astronomy"
    
    #fix this one...
    if (nme == "Alpha Centauri"):
        search = nme
        
    site = wikipedia.search(search)    
    print(search)
    if (site):
        print("wiki page: ", site[0])
        page = None
        try:
            page = wikipedia.page(site[0])
        except wikipedia.exceptions.DisambiguationError as e:
            print(e.options)
            #pass
        if (page):
            images = []
            captions = []
            
            #get more info with wptools
            #https://github.com/siznax/wptools/wiki/
            wpage = wptools.page(site[0])
            wpage.get_more()
            wpage.get_query()
            wpage.get_parse()
            
            #get all the images (works better with wikipedia instead of wptools, but need wptools for captions)
            #print(page.images)
            #print(wpage.data['parsetree'])
            for i in page.images:
                if (("jpg" in i or "png" in i) and "Celestia" not in i):
                    cap = getCaption(wpage.data['parsetree'], i)
                    if (cap): #only keep images with captions.  I think this will cut down on the images we don't want.
                        captions.append(cap)
                        images.append(i)
                    
        
            #get the main image (only possible with wptools)
            mainImg = None
            mainCap = None
            if (wpage.images()):
                for i in wpage.images():
                    print('CHECKING :', i['kind'])
                    if (i['kind'] == 'query-pageimage'):
                        mainImg = i['url']
                        print('HAVE mainImg : ', mainImg)
                        if ('metadata' in i):
                            if ('ImageDescription' in i['metadata']):
                                mainCap = i['metadata']['ImageDescription']['value']
                                
                        if (mainCap == None):
                            mainCap = getCaption(wpage.data['parsetree'], i['url'])
                        print('HAVE mainCap :', mainCap)
                            
            if (mainImg):
                img2 = images
                caption2 = captions
                images = [mainImg]
                captions = [mainCap]
                for i,c in zip(img2,caption2):
                    if (i != mainImg):
                        images.append(i)
                        captions.append(c)
            

                
            return images, captions, page.url

    return None, None, None

In [3]:
files = ['TileWallGalaxies.csv', 'TileWallGalaxyClusters.csv','TileWallNebulae.csv', \
         'TileWallStarClusters.csv','TileWallStars.csv',  'TileWallConstellations.csv' ]
tpe = ['Galaxies', 'Galaxy Clusters', 'Nebulae', 'Star Clusters', 'Stars', 'Constellations']
#not sure what to do about the 'Constellation Detailed Notes'...

In [4]:
for i,f in enumerate(files):
    df = pd.read_csv(f, encoding = "ISO-8859-1")
    for index, row in df.iterrows():
        if (isinstance(row['Object'], str)):
            row['Category'] = tpe[i]
            d = {}
            nme = row['Object']
            fname = os.path.join('objectFiles',''.join(e for e in nme if e.isalnum())+'.json')
            print(fname)
            
            d[nme] = row
            d[nme].pop('Object')
            imgurls, captions, url = getWikiImages(nme, tpe[i])
            d[nme]['images'] = None
            d[nme]['captions'] = None
            d[nme]['wikipedia'] = None
            if (imgurls):
                d[nme]['images'] = imgurls
                d[nme]['captions'] = captions
                d[nme]['wikipedia'] = url
                print(imgurls[0])
            pd.DataFrame(d).to_json(fname)
            print("")

objectFiles/AndromedaM31.json
Andromeda M31  
wiki page:  Andromeda Galaxy


en.wikipedia.org (querymore) Andromeda Galaxy
en.wikipedia.org (querymore) 74331 (&blcontinue=0|6390647)
en.wikipedia.org (querymore) 74331 (&blcontinue=3|38333229)
en.wikipedia.org (querymore) 74331 (&blcontinue=0|55515710)
Andromeda Galaxy (en) data
{
  backlinks: <list(1622)> {'pageid': 1925, 'ns': 0, 'title': 'Andr...
  categories: <list(12)> Category:Andromeda (constellation), Categ...
  contributors: 1,066
  files: <list(28)> File:A Swift Tour of M31.OGG, File:A Swift Tou...
  languages: <list(94)> {'lang': 'af', 'title': 'Andromeda-sterres...
  pageid: 74331
  requests: <list(4)> querymore, querymore, querymore, querymore
  title: Andromeda Galaxy
  views: 2,092
}
en.wikipedia.org (query) Andromeda Galaxy
en.wikipedia.org (query) Andromeda Galaxy (&plcontinue=74331|0|NGC_147)
en.wikipedia.org (query) Andromeda Galaxy (&plcontinue=74331|0|Pa...
en.wikipedia.org (imageinfo) File:Andromeda Galaxy (with h-alpha).jpg
Andromeda Galaxy (en) data
{
  WARNINGS: <dict(1)> extracts
  alias


SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/9/92/Andromeda_active_core.jpg Andromeda active core.jpg
CAPTION :  Artist's concept of the Andromeda Galaxy core showing a view across a disk of young, blue stars encircling a supermassive black hole. NASA/ESA&amp;nbsp;photo.

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/f/ff/Andromeda_galaxy.jpg Andromeda galaxy.jpg
CAPTION :  The Andromeda Galaxy pictured in ultraviolet light by GALEX.

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/0/0d/Andromeda_galaxy_2.jpg Andromeda galaxy 2.jpg
CAPTION :  

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/a/a4/Charles_Messier.jpg Charles Messier.jpg
CAPTION :  

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/c/c5/Hubble_Finds_Giant_Halo_Around_the_Andromeda_Galaxy.jpg Hubble Finds Giant Halo Around the Andromeda Galaxy.jpg
CAPTION :  Giant halo around Andromeda Galaxy.

SEARCHING FOR :  https://upload.wikimedia.or

en.wikipedia.org (querymore) Antennae Galaxies
Antennae Galaxies (en) data
{
  backlinks: <list(362)> {'pageid': 11971, 'ns': 0, 'title': 'Gala...
  categories: <list(12)> Category:Arp objects, Category:Astronomic...
  contributors: 169
  files: <list(9)> File:Antennae Galaxies NGC4038 NGC4039 Goran Ni...
  languages: <list(35)> {'lang': 'af', 'title': 'Antenne-sterreste...
  pageid: 1111758
  requests: <list(1)> querymore
  title: Antennae Galaxies
  views: 112
}
en.wikipedia.org (query) Antennae Galaxies
en.wikipedia.org (query) Antennae Galaxies (&plcontinue=1111758|0...
en.wikipedia.org (imageinfo) File:Antennae Galaxies reloaded.jpg
Antennae Galaxies (en) data
{
  WARNINGS: <dict(1)> extracts
  assessments: <dict(1)> Astronomy
  backlinks: <list(362)> {'pageid': 11971, 'ns': 0, 'title': 'Gala...
  categories: <list(12)> Category:Arp objects, Category:Astronomic...
  contributors: 169
  description: interacting galaxies in the constellation Corvus
  extext: <str(344)> The **Antenna


SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/a/a0/Antennae_Galaxies_NGC4038_NGC4039_Goran_Nilsson_%26_The_Liverpool_Telescop.jpg Antennae Galaxies NGC4038 NGC4039 Goran Nilsson %26 The Liverpool Telescop.jpg
CAPTION :  

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/0/03/Antennae_Galaxies_composite_of_ALMA_and_Hubble_observations.jpg Antennae Galaxies composite of ALMA and Hubble observations.jpg
bad caption :  2493
CAPTION :  None

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/3/30/Antennae_Galaxies_reloaded.jpg Antennae Galaxies reloaded.jpg
bad caption :  5662
CAPTION :  None

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/5/5d/VLT_Observes_the_Antennae_Galaxies.jpg VLT Observes the Antennae Galaxies.jpg
bad caption :  2493
CAPTION :  None
CHECKING : query-pageimage
HAVE mainImg :  https://upload.wikimedia.org/wikipedia/commons/3/30/Antennae_Galaxies_reloaded.jpg
HAVE mainCap : The NASA/ESA Hubble Space Telesco

...


SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/b/b9/Big_dipper.triddle.jpg Big dipper.triddle.jpg
bad caption :  6
CAPTION :  None

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/5/5d/Polarbear_spitzbergen_1.jpg Polarbear spitzbergen 1.jpg
CAPTION :  H. A. Rey's alternative asterism for Ursa Major can be said to give it the longer head and neck of a polar bear, as seen in this photo, from the ''left'' side.

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/9/94/Starry_Night_Over_the_Rhone.jpg Starry Night Over the Rhone.jpg
bad caption :  4353
CAPTION :  None

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/8/86/UrsaMajorCC.jpg UrsaMajorCC.jpg
CAPTION :  The constellation Ursa Major as it can be seen by the unaided eye.

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/c/ce/Ursa_Major_-_Ursa_Minor_-_Polaris.jpg Ursa Major - Ursa Minor - Polaris.jpg
CAPTION :  Ursa Major and Ursa Minor in relation to Polari

en.wikipedia.org (querymore) Ursa Minor
en.wikipedia.org (querymore) 31773 (&blcontinue=0|51145663)
Ursa Minor (en) data
{
  backlinks: <list(546)> {'pageid': 798, 'ns': 0, 'title': 'Aries ...
  categories: <list(4)> Category:Constellations, Category:Constell...
  contributors: 633
  files: <list(9)> File:Blue pencil.svg, File:Celestia.png, File:C...
  languages: <list(92)> {'lang': 'af', 'title': 'Klein Beer'}, {'l...
  pageid: 31773
  requests: <list(2)> querymore, querymore
  title: Ursa Minor
  views: 463
}
en.wikipedia.org (query) Ursa Minor
en.wikipedia.org (imageinfo) File:Ursa Minor IAU.svg
Ursa Minor (en) data
{
  WARNINGS: <dict(1)> extracts
  aliases: <list(4)> Little Bear, Little Dipper, UMi, Ursae Minoris
  assessments: <dict(1)> Astronomy
  backlinks: <list(546)> {'pageid': 798, 'ns': 0, 'title': 'Aries ...
  categories: <list(4)> Category:Constellations, Category:Constell...
  contributors: 633
  description: <str(90)> constellation in the northern celestial h...
  extex


SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/8/89/NGC_6217_hs-2009-25-bc-full_jpg.jpg NGC 6217 hs-2009-25-bc-full jpg.jpg
bad caption :  8
CAPTION :  None

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/7/72/UrsaMinorCC.jpg UrsaMinorCC.jpg
CAPTION :  The constellation Ursa Minor as it can be seen by the naked eye (with connections and label added).

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/c/ce/Ursa_Major_-_Ursa_Minor_-_Polaris.jpg Ursa Major - Ursa Minor - Polaris.jpg
CAPTION :  Ursa Minor and Ursa Major in relation to Polaris

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/3/3f/Sidney_Hall_-_Urania%27s_Mirror_-_Draco_and_Ursa_Minor.jpg Sidney Hall - Urania%27s Mirror - Draco and Ursa Minor.jpg
CAPTION :  
CHECKING : query-pageimage
HAVE mainImg :  https://upload.wikimedia.org/wikipedia/commons/c/cb/Ursa_Minor_IAU.svg
HAVE mainCap : IAU Ursa Minor chart
CHECKING : query-thumbnail
https://upload.wikimedia.org/

en.wikipedia.org (querymore) Virgo (constellation)
en.wikipedia.org (querymore) 32407 (&blcontinue=3|31466683)
Virgo (constellation) (en) data
{
  backlinks: <list(796)> {'pageid': 798, 'ns': 0, 'title': 'Aries ...
  categories: <list(4)> Category:Constellations, Category:Constell...
  contributors: 609
  files: <list(7)> File:Celestia.png, File:Commons-logo.svg, File:...
  languages: <list(87)> {'lang': 'af', 'title': 'Maagd (sterrebeel...
  pageid: 32407
  requests: <list(2)> querymore, querymore
  title: Virgo (constellation)
  views: 500
}
en.wikipedia.org (query) Virgo (constellation)
en.wikipedia.org (query) Virgo (constellation) (&plcontinue=32407...
en.wikipedia.org (query) Virgo (constellation) (&plcontinue=32407...
en.wikipedia.org (imageinfo) File:Virgo IAU.svg
Virgo (constellation) (en) data
{
  WARNINGS: <dict(1)> extracts
  aliases: <list(2)> Vir, Virginis
  assessments: <dict(1)> Astronomy
  backlinks: <list(796)> {'pageid': 798, 'ns': 0, 'title': 'Aries ...
  categories


SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/3/3a/F16.r._Virgo_-_NLW_MS_735C.png F16.r. Virgo - NLW MS 735C.png
CAPTION :  Depiction of Virgo, c.1000

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/3/37/VirgoCC.jpg VirgoCC.jpg
CAPTION :  The constellation Virgo.

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/f/f2/When_Stars_Explode_NGC_4981.jpg When Stars Explode NGC 4981.jpg
CAPTION :  NGC 4981 was discovered on 17 April 1784 by William Herschel.

SEARCHING FOR :  https://upload.wikimedia.org/wikipedia/commons/3/38/Sidney_Hall_-_Urania%27s_Mirror_-_Virgo.jpg Sidney Hall - Urania%27s Mirror - Virgo.jpg
CAPTION :  
CHECKING : query-pageimage
HAVE mainImg :  https://upload.wikimedia.org/wikipedia/commons/f/f5/Virgo_IAU.svg
HAVE mainCap : IAU Virgo chart
CHECKING : query-thumbnail
CHECKING : query-pageimage
HAVE mainImg :  https://upload.wikimedia.org/wikipedia/commons/f/f5/Virgo_IAU.svg
HAVE mainCap : IAU Virgo chart
CHECKING : query